# Import the modules needed, defining paths and funtions  

In [ ]:
import os
import sys
import csv
import time
import glob
import json
import requests
import re
import shutil
import subprocess
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import numpy as np
import pickle as pkl
import pandas as pd
from pathlib import Path
from datetime import datetime as dt
from fpdf import FPDF 
from fpdf.enums import XPos, YPos
#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('basic_functions.py'), '..')))
from basic_functions import (cs8_os, c7_os, directory, get_unix_timestamp, make_column_list, datenum, is_hidden, check_OS, 
                             make_file_list, make_name_list, get_column_val, break_file_name, get_query_urls, json_info, 
                             extract_data_and_stats_from_panel, uprof_pcm_formatter, convert, uprof_timechart_formatter, 
                             month2num, add_new_time_format, fetch_grafana_panels, create_var_list, debug)

columns_list_CPU = [['C0 Core C-state residency'], ['CPU Utilization']]
label_CPU = 'CPU Utilization (%)'

pcm_power_columns_list_0 = ['Package Joules Consumed Socket0 Energy Consumption']
pcm_power_columns_list_1 = ['Package Joules Consumed Socket1 Energy Consumption']
uprof_power_columns_list_0 = ['socket0-package-power']
uprof_power_columns_list_1 = ['socket1-package-power']
label_POWER = 'CPU Power Consumption (Watt)'

pcm_columns_list_0 = ['Socket0 Memory Bandwidth', 'Socket0 Instructions Per Cycle', 'Socket0 L2 Cache Misses', 'Socket0 L3 Cache Misses']
pcm_columns_list_1 = ['Socket1 Memory Bandwidth', 'Socket1 Instructions Per Cycle', 'Socket1 L2 Cache Misses', 'Socket1 L3 Cache Misses']
uprof_columns_list_0 = ['Total Mem Bw (GB/s) Socket0', 'IPC (Sys + User) Socket0', 'L2 Miss (pti) Socket0', 'L3 Miss Socket0']
uprof_columns_list_1 = ['Total Mem Bw (GB/s) Socket1', 'IPC (Sys + User) Socket1', 'L2 Miss (pti) Socket1', 'L3 Miss Socket1']
label_names = ['Memory Bandwidth (GB/sec)', 'Instructions Per Cycle', 'L2 Cache Misses (Million)', 'L3 Cache Misses (Million)']

label_columns = ['Socket0', 'Socket1'] 
color_list = ['red', 'blue', 'green', 'cyan', 'orange', 'navy', 'magenta', 'lime', 'purple', 'hotpink', 
              'olive', 'salmon', 'teal', 'darkblue', 'darkgreen', 'darkcyan', 'darkorange', 'deepskyblue', 
              'darkmagenta', 'sienna', 'chocolate', 'orangered', 'gray', 'royalblue', 'gold', 'peru', 
              'seagreen', 'violet', 'tomato', 'lightsalmon', 'crimson', 'lightblue', 'lightgreen', 
              'lightpink', 'black', 'darkgray', 'lightgray', 'saddlebrown', 'brown', 'khaki', 'tan', 
              'turquoise', 'linen', 'lawngreen']
linestyle_list = ['solid', 'dotted', 'dashed', 'dashdot','solid', 'dotted', 'dashed', 'dashdot']
marker_list = ['s','o','.','p','P','^','<','>','*','+','x','X','d','D','h','H']

def plot_vars_comparison(input_dir, output_dir, pdf_name):
    X_plot, X_plot_power, Y_plot, Y_plot_0, Y_plot_1, Y_plot_power_0, Y_plot_power_1 = [], [], [], [], [], [], []
    label_plot, label_plot_0, label_plot_1, label_plot_power_0, label_plot_power_1 = [], [], [], [], []
    
    pcm_file, uprof_file, time_file, reformated_uprof_file, reformated_time_file, all_file, all_plots_file, all_plots_power_list = make_name_list(input_dir)
    
    for i, file_i in enumerate(all_plots_file):    
        info = break_file_name(file_i)
        data_frame = pd.read_csv('{}/{}.csv'.format(input_dir, file_i))
        X_plot.append(data_frame['NewTime'].values.tolist())
                
        Y_tmp, Y_tmp_0, Y_tmp_1, Y_tmp_power_0, Y_tmp_power_1 = [], [], [], [], []
        label_tmp, label_tmp_0, label_tmp_1, label_tmp_power_0, label_tmp_power_1 = [], [], [], [], []
        
        if info[0]=='grafana':
            for j, columns_pcm in enumerate(columns_list_CPU[0]):
                Y, label = get_column_val(data_frame, [columns_pcm], label_columns, file_i) 
                Y_tmp.append(Y)
                label_tmp.append(label)  
        else:    
            for j, columns_uprof in enumerate(columns_list_CPU[1]):
                Y, label = get_column_val(data_frame, [columns_uprof], label_columns, file_i)
                Y_tmp.append(Y)
                label_tmp.append(label)
            
        Y_plot.append(Y_tmp)
        label_plot.append(label_tmp)
        
        if info[0]=='grafana':
            for k, (columns_pcm_0, columns_pcm_1) in enumerate(zip(pcm_columns_list_0, pcm_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_pcm_0], [label_columns[0]], file_i)  
                Y_1, label_1 = get_column_val(data_frame, [columns_pcm_1], [label_columns[1]], file_i)  
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
        else:
            for k, (columns_uprof_0, columns_uprof_1) in enumerate(zip(uprof_columns_list_0, uprof_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_uprof_0], [label_columns[0]], file_i)
                Y_1, label_1 = get_column_val(data_frame, [columns_uprof_1], [label_columns[1]], file_i)
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
    
        Y_plot_0.append(Y_tmp_0)
        label_plot_0.append(label_tmp_0)
        Y_plot_1.append(Y_tmp_1)
        label_plot_1.append(label_tmp_1)
        
    for i, file_i in enumerate(all_plots_power_list):  
        info = break_file_name(file_i)
        data_frame = pd.read_csv('{}/{}.csv'.format(input_dir, file_i))
        X_plot_power.append(data_frame['NewTime'].values.tolist())
        
        if info[0]=='grafana':
            for k, (columns_pcm_0, columns_pcm_1) in enumerate(zip(pcm_power_columns_list_0, pcm_power_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_pcm_0], [label_columns[0]], file_i)  
                Y_1, label_1 = get_column_val(data_frame, [columns_pcm_1], [label_columns[1]], file_i)  
                Y_tmp_power_0.append(Y_0)
                label_tmp_power_0.append(label_0)
                Y_tmp_power_1.append(Y_1)
                label_tmp_power_1.append(label_1)
        else:
            for k, (columns_uprof_0, columns_uprof_1) in enumerate(zip(uprof_power_columns_list_0, uprof_power_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_uprof_0], [label_columns[0]], file_i)
                Y_1, label_1 = get_column_val(data_frame, [columns_uprof_1], [label_columns[1]], file_i)
                Y_tmp_power_0.append(Y_0)
                label_tmp_power_0.append(label_0)
                Y_tmp_power_1.append(Y_1)
                label_tmp_power_1.append(label_1)
    
        Y_plot_power_0.append(Y_tmp_power_0)
        label_plot_power_0.append(label_tmp_power_0)
        Y_plot_power_1.append(Y_tmp_power_1)
        label_plot_power_1.append(label_tmp_power_1)
        
    # Here we make the plot:
    matplotlib.rcParams['font.family'] = 'DejaVu Serif'
    #-----------------------------------
    plt.figure(figsize=(16, 5))
    plt.style.use('default')
    
    for i in range(len(Y_plot)):
        plt.plot(X_plot[i], Y_plot[i][0][0], color=color_list[i], label=label_plot[i][0][0])
        plt.ylabel(label_CPU)
        plt.xlabel('Time (min)')
        plt.grid(which='major', color='gray', linestyle='dashed')
        plt.legend(loc='upper left')
        
    plt.tight_layout()
    plt.savefig('{}/{}_comparison_{}_{}_CPU.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    #-----------------------------------
    plt.figure(figsize=(16, 5))
    plt.style.use('default')
    
    for i in range(len(Y_plot)):
        plt.plot(X_plot_power[i], Y_plot_power_0[i][0][0], color=color_list[i], label=label_plot_power_0[i][0][0])
        plt.ylabel(label_POWER)
        plt.xlabel('Time (min)')
        plt.grid(which='major', color='gray', linestyle='dashed')
        plt.legend(loc='upper left')
        
    plt.tight_layout()
    plt.savefig('{}/{}_comparison_{}_{}_POWER_socket0.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    #-----------------------------------
    plt.figure(figsize=(16, 5))
    plt.style.use('default')
    
    for i in range(len(Y_plot)):
        plt.plot(X_plot_power[i], Y_plot_power_1[i][0][0], color=color_list[i], label=label_plot_power_1[i][0][0])
        plt.ylabel(label_POWER)
        plt.xlabel('Time (min)')
        plt.grid(which='major', color='gray', linestyle='dashed')
        plt.legend(loc='upper left')
        
    plt.tight_layout()
    plt.savefig('{}/{}_comparison_{}_{}_POWER_socket1.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    #-----------------------------------
    fig, axs = plt.subplots(2, 2, figsize=(16, 10))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_0)):  #number of files or tests
        for j in range(len(Y_plot_0[i])):  #number of metrix
            axs[j].plot(X_plot[i], Y_plot_0[i][j][0], color=color_list[i], label=label_plot_0[i][j][0], linestyle=linestyle_list[0])
            axs[j].set_ylabel('{}'.format(label_names[j]))
            axs[j].set_xlabel('Time (min)')
            axs[j].grid(which='major', color='gray', linestyle='dashed')
            axs[j].legend(loc='upper left')
               
    plt.tight_layout()
    plt.savefig('{}/{}_comparison_{}_{}_socket0.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    #-----------------------------------
    fig, axs = plt.subplots(2, 2, figsize=(16, 10))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_1)):  
        for j in range(len(Y_plot_1[i])):
            axs[j].plot(X_plot[i], Y_plot_1[i][j][0], color=color_list[i], label=label_plot_1[i][j][0], linestyle=linestyle_list[0])
            axs[j].set_ylabel('{}'.format(label_names[j]))
            axs[j].set_xlabel('Time (min)')
            axs[j].grid(which='major', color='gray', linestyle='dashed')
            axs[j].legend(loc='upper left')
            
    plt.tight_layout()
    plt.savefig('{}/{}_comparison_{}_{}_socket1.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
def create_report_performance(input_dir, output_dir, daqconfs_cpupins_folder_parent_dir, process_pcm_files=False, process_uprof_files=False, print_info=True, streams='8, 16, 24, 32, 40, and 48', pdf_name='performance_report'):    
    directory([input_dir, output_dir])
    now = dt.now()
    current_dnt = now.strftime('%Y-%m-%d %H:%M:%S')
    
    pcm_file, uprof_file, time_file, reformated_uprof_file, reformated_time_file, all_file, all_plots_file, all_plots_power_list = make_name_list(input_dir)
    
    # Open pdf file
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Times', 'B', 16)
    pdf.cell(40,10,'Performance Report')
    pdf.write(5,'\n\n')
    
    # Processing the data first
    if process_pcm_files:
        for i, file_pcm_i in enumerate(pcm_file):
            add_new_time_format(input_dir, file_pcm_i)

    if process_uprof_files:
        for i, (file_time_i, file_uprof_i) in enumerate(zip(time_file, uprof_file)):
            uprof_timechart_formatter(input_dir, file_time_i)
            uprof_pcm_formatter(input_dir, file_uprof_i)
            #combine_time_and_uprof_files(input_dir, file_time_i, file_uprof_i)
            add_new_time_format(input_dir, 'reformatter_{}'.format(file_uprof_i))
            add_new_time_format(input_dir, 'reformatter_{}'.format(file_time_i))
    
    if process_pcm_files or process_uprof_files:
        print('Finish the processing of the data.')
    
    info_pcm_basic = break_file_name(all_file[0])
    
    # creating report
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The tests were run using the dunedaq version fddaq-{} and for the WIB{} data format for {} streams. The Figures show the comparison of the tests ran (Table) using the different metrics. \n'.format(info_pcm_basic[1], info_pcm_basic[4], streams))
    pdf.write(5, '    * L2-misses is the fraction of requests that make it to L2 at all and then miss in L2. Similar for L3. \n')
    pdf.write(5, '    * AMD Power is the Average Power for the sampling period, reported in Watts. This is an estimated consumption value based on platform activity levels. \n')      
    pdf.write(5,'\n\n')
    #-------------------------------------------TABLE-----------------------------------------------
    data = []
    headers = ['Test', 'Readout app SRV', 'OS', 'NODE', 'Other app SRV']
    data.append(headers)
    
    line_height = pdf.font_size * 2
    col_width = [pdf.epw/3.8, pdf.epw/6.5, pdf.epw/10, pdf.epw/10, pdf.epw/7]
    
    lh_list = [] #list with proper line_height for each row
    use_default_height = 0 #flag
    
    for i, file_i in enumerate(all_file):
        info = break_file_name(file_i)
        line = [info[5], info[2], check_OS(info[2]), info[3], info[6]]
        data.append(line)
    
    for row in data:
        for datum in row:
            word_list = datum.split()
            number_of_words = len(word_list) #how many words
            if number_of_words>2: 
                use_default_height = 1
                new_line_height = pdf.font_size * (number_of_words) #new height change according to data 
        if not use_default_height:
            lh_list.append(line_height)
        else:
            lh_list.append(new_line_height)
            use_default_height = 0
        
    for j, row in enumerate(data):
        for k, datum in enumerate(row):
            line_height = lh_list[j] #choose right height for current row
            pdf.multi_cell(col_width[k], line_height, datum, border=1, align='L', new_x=XPos.RIGHT, new_y=YPos.TOP, max_line_height=pdf.font_size)
            
        pdf.ln(line_height)
        
    pdf.write(5, 'Table. Summary of the tests ran. \n')    
    pdf.write(5,'\n\n')
    #--------------------------------------------FIGURES------------------------------------------------
    plot_vars_comparison(input_dir, output_dir, pdf_name)
    pdf.image('{}/performance_comparison_{}_{}_CPU.png'.format(output_dir, info_pcm_basic[1], info_pcm_basic[4]), w=180)
    pdf.write(5, 'Figure. Comparison of the "CPU Utilization" for the tests ran.')
    pdf.write(5,'\n\n')
    
    if info[3] == '0' or info[3] == '2':
        pdf.image('{}/performance_comparison_{}_{}_socket0.png'.format(output_dir, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure. Socket0 comparison of the tests ran using the metrics "Memory Bandwidth", "Instructions Per Cycle", "L2 Cache Misses", "L3 Cache Misses".')
        pdf.write(5,'\n\n')
        
    if info[3] == '1' or info[3] == '2':
        pdf.image('{}/performance_comparison_{}_{}_socket1.png'.format(output_dir, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure. Socket1 comparison of the tests ran using the metrics "Memory Bandwidth", "Instructions Per Cycle", "L2 Cache Misses", "L3 Cache Misses".')
        pdf.write(5,'\n\n')
     
    if info[3] == '0' or info[3] == '2':
        pdf.image('{}/performance_comparison_{}_{}_POWER_socket0.png'.format(output_dir, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure. Socket0 comparison of the "CPU Power Consumption" for the tests ran.')
        pdf.write(5,'\n\n')
        
    if info[3] == '1' or info[3] == '2':
        pdf.image('{}/performance_comparison_{}_{}_POWER_socket1.png'.format(output_dir, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure. Socket1 comparison of the "CPU Power Consumption" for the tests ran.')
        pdf.write(5,'\n\n')
    
    #----------------------------------------CONFIGURATIONS---------------------------------------------
    pdf.write(5,'\n\n')
    
    if print_info:
        pdf.write(5, 'Configurations: \n', 'B')
        for i, file_i in enumerate(all_file):
            info = break_file_name(file_i)
            json_info(file_daqconf='daqconf-{}-{}-{}'.format(info[4], info[5], info[2]), input_dir=daqconfs_cpupins_folder_parent_dir, var='ru{}{}{}'.format(info[2], info[4], info[3]), pdf=pdf, if_pdf=print_info)

    pdf.write(5, '\n\n\n')
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The End, made on {}'.format(current_dnt))
    pdf.output('{}/{}_report.pdf'.format(output_dir, pdf_name))
            
print('Ready to run and process')

## Proccesing data from Grafana
Note: change the paths to fit yours

To extract the data from a given dashboard in grafana:
* 'grafana_url' is:
    * 'http://np04-srv-009.cern.ch:3000'  (legacy)
    * 'http://np04-srv-017.cern.ch:31023' (new) 
* 'dashboard_uid' is the unic dashboard identifiyer, you can find this information on the link of the dashboard. The dashboard_uid code is in the web link after/d/.../ 
    * for intel-r-performance-counter-monitor-intel-r-pcm dashboard dashboard_uid = '91zWmJEVk' 
    * for = 'v4_2_0-trigger_primitives'
* delta_time is [start, end] given in the format '%Y-%m-%d %H:%M:%S'
* host is the name of the server in study for example: "np02-srv-003"     

file_name (for performance tests): [version]-[server_app_tested]-[numa node]-[data format]-[tests_name]-[server rest_of the apps]
* example of name: v4_1_1-np02srv003-0-eth-stream_scaling-np04srv003

In [ ]:
grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 00:00:00', '2023-11-28 00:00:00'],
              ['2023-11-28 00:00:00', '2023-11-28 00:00:00']]
output_csv_file = ['v4_2_0-np02srv003-0-eth-stream_scaling-np04srv025', 
                   'v4_2_0-np02srv003-0-eth-stream_scaling_swtpg-np04srv025']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 00:00:00', '2023-11-28 00:00:00'],
              ['2023-11-28 00:00:00', '2023-11-28 00:00:00']]
output_csv_file = ['v4_2_0-np02srv003-0-eth-stream_scaling_recording-np04srv025', 
                   'v4_2_0-np02srv003-0-eth-stream_scaling_recording_swtpg-np04srv025']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, input_dir=results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
tpg_delta_time = [['2023-11-23 18:41:24', '2023-11-23 18:56:32'],
                  ['2023-11-23 18:58:48', '2023-11-23 19:13:56'],
                  ['2023-11-24 13:15:46', '2023-11-24 13:30:54'],
                  ['2023-11-24 13:55:35', '2023-11-24 14:10:44'],
                  ['2023-11-24 11:55:41', '2023-11-24 12:10:50']]
tpg_output_csv_file = ['v4_2_0-np02srv003-0-eth-48streams_swtpg-np04srv025', 
                       'v4_2_0-np02srv003-1-eth-48streams_swtpg-np04srv025', 
                       'v4_2_0-np02srv003-0-eth-48streams_recording_swtpg-np04srv025',
                       'v4_2_0-np02srv003-1-eth-48streams_recording_swtpg-np04srv025',
                       'v4_2_0-np02srv003-1-eth-48streams_recording_swtpg_multinode-np04srv025']
tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_intel'

for delta_time_list, output_csv_file_list in zip(tpg_delta_time, tpg_output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, 
                                      host=host_used, input_dir=tpg_results_path, 
                                      output_csv_file=output_csv_file_list)

print('done :-)')

## Performance report
Note: change the paths to fit yours

In [ ]:
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='8, 16, 24, 32, 40, and 48', 
                          pdf_name='performancetest_without_recording')

print('THE END')

In [ ]:
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='8, 16, 24, 32, 40, and 48', 
                          pdf_name='performancetest_with_recording_report')

print('THE END')

In [ ]:
#Performance report for 48 streams only, to study the TPG rate problems in the intel

tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_intel'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=tpg_results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=False, 
                          print_info=True, streams='48', pdf_name='performancetest_tpg_intel_report')

print('THE END')

In [ ]:
#Performance report for 48 streams only, to study the TPG rate problems in the amd

tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_amd'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=tpg_results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=False, process_uprof_files=True, 
                          print_info=True, streams='48', pdf_name='performancetest_tpg_amd_report')

print('THE END')

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('basic_functions.py'), '..')))